In [24]:
import torch
import Models.pytorch_joy_and_anger.joy_and_anger_utils as model_utils

In [25]:
train_ds = model_utils.HappyClassifierDataset("train.txt", probabilistic=True)
test_ds = model_utils.HappyClassifierDataset("test.txt", probabilistic=True)

loaded 7520 items
{'joy': 0.0, 'anger': 1.0}
('im grabbing a minute to post i feel greedy wrong', 1)
loaded 970 items
{'joy': 0.0, 'anger': 1.0}
('i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived', 0)


In [26]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [27]:
tokenizer = get_tokenizer('basic_english')

In [28]:
vocab = build_vocab_from_iterator(list(map(lambda k: tokenizer(k), [txt for txt, label in train_ds.train_data])), specials=["<unk>"])

In [29]:
vocab.set_default_index(vocab["<unk>"])

In [30]:
vocab(['great', 'day', "we're", 'having'])

[353, 96, 0, 171]

In [31]:
# idx 2 has issues

In [32]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: x

In [33]:
from torch.utils.data import DataLoader
device = torch.device("cpu")

def collate_batch(batch):
    text_list, label_list, offsets = [], [], [0]
    for text, label in batch:
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
        label_list.append(label_pipeline(label))
        offsets.append(processed_text.size(0))
    text_list = torch.cat(text_list)
    label_list = torch.tensor(label_list, dtype=torch.float32)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    return text_list.to(device), label_list.to(device), offsets.to(device)

PyTorch seems to have a LSTM layer (mostly) similar to tf2 so we don't have to create it from scratch here.

In [34]:
from torch import nn

#TODO: Build a Backward RNN forward pass
class HappyClassifierModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(HappyClassifierModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=64, bidirectional=True)
        self.linear1 = nn.Linear(64, 32)

        self.fc1 = nn.Sequential(
            self.linear1,
            nn.ReLU()
        )
        self.linear2 = nn.Linear(32, 1)
        self.fc2 = nn.Sequential(
            self.linear2
        )

        # self.fc = nn.Sequential(
        #     self.linear1,
        #     nn.ReLU(),
        #     self.linear2
        # )

        self.init_weights()

    def init_weights(self):
        initrange = 0.2
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear1.weight.data.uniform_(-initrange, initrange)
        self.linear2.weight.data.uniform_(-initrange, initrange)
        self.linear1.bias.data.zero_()
        self.linear2.bias.data.zero_()

    def forward(self, text, offsets):
        layer = self.embedding(text, offsets)
        layer = self.fc1(layer)
        layer = self.fc2(layer)
        return layer

In [35]:
num_class = 2
vocab_size = len(vocab)
emsize = 64
model = HappyClassifierModel(vocab_size, emsize, num_class).to(device)

In [36]:
from torchviz import make_dot

In [37]:
train_dataloader = DataLoader(train_ds, batch_size=64, shuffle=False, collate_fn=collate_batch)

In [38]:
next(iter(train_dataloader))

(tensor([  14, 6945,    6,  ...,  119,    2,   94]),
 tensor([1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([   0,   10,   14,   33,   54,   66,  108,  118,  130,  139,  166,  184,
          205,  214,  226,  260,  305,  317,  326,  351,  380,  407,  430,  457,
          484,  502,  566,  572,  580,  592,  597,  630,  660,  679,  702,  741,
          753,  769,  794,  832,  845,  862,  867,  889,  912,  921,  933,  959,
          981,  997, 1014, 1031, 1048, 1063, 1104, 1113, 1133, 1156, 1177, 1221,
         1257, 1266, 1296, 1301]))

Verify our data can go through the model

In [39]:
for i, (txt, label, offset) in enumerate(train_dataloader):
    print(label.shape)
    print(label[0])
    print(txt, label, offset)
    print(model(txt, offset))
    break

torch.Size([64])
tensor(1.)
tensor([  14, 6945,    6,  ...,  119,    2,   94]) tensor([1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([   0,   10,   14,   33,   54,   66,  108,  118,  130,  139,  166,  184,
         205,  214,  226,  260,  305,  317,  326,  351,  380,  407,  430,  457,
         484,  502,  566,  572,  580,  592,  597,  630,  660,  679,  702,  741,
         753,  769,  794,  832,  845,  862,  867,  889,  912,  921,  933,  959,
         981,  997, 1014, 1031, 1048, 1063, 1104, 1113, 1133, 1156, 1177, 1221,
        1257, 1266, 1296, 1301])
tensor([[-2.6045e-02],
        [-5.9599e-03],
        [ 2.9444e-03],
        [-9.9295e-03],
        [ 1.4744e-02],
        [-4.4782e-03],
        [ 1.8922e-03],
        [ 2.0264e-02],
        [-3.150

In [40]:
# lets try these hyperparameters, losses, and optimizers instead and see our results

epochs = 10
LR = 5
BATCH_SIZE = 64

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1.0, gamma=0.1)
total_accu = None

In [41]:
# redfine our datasets

train_dataloader = DataLoader(dataset=train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(dataset=test_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [42]:
import time

def train(dataloader):
    model.train()
    total_accuracy, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for i, (texts, labels, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        pred = model(texts, offsets)
        labels = torch.tensor([[a.type(torch.LongTensor)] for a in labels], dtype=torch.float)
        loss = criterion(pred, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        #our result is the largest number's index of the prediction
        total_accuracy += (pred.argmax(1) == labels).sum().item()
        total_count += labels.size(0)

        if i % log_interval == 0 and i > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, i, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (texts, labels, offsets) in enumerate(dataloader):
            pred = model(texts, offsets)
            labels = torch.tensor([[a.type(torch.LongTensor)] for a in labels], dtype=torch.float)
            loss = criterion(pred, labels)
            total_acc += (pred.argmax(1) == labels).sum().item()
            total_count += labels.size(0)
    return total_acc/total_count





In [43]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(test_dataloader)

    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

RuntimeError: all elements of input should be between 0 and 1

In [ ]:
a = torch.tensor(0.6, dtype=torch.float32)

In [ ]:
a.argmax(0)

In [ ]:
predicted_label = torch.tensor([0.2, 0.3, 0.3, 0.3])
label = 1.
a = (predicted_label.argmax(1) == label)
b = a.sum()
c = b.item()
print(a,b,c)